# Trained the pretrained weight

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
# os.environ["CUDA_VISIBLE_DEVICES"] = '3'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda, Dropout
from keras.layers import LeakyReLU, Multiply, add, dot, SeparableConv2D, DepthwiseConv2D, Reshape, Add, ReLU, Permute, Layer, InputSpec
from keras.regularizers import l2
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras import regularizers
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
# from tqdm import tqdm, trange
import random

Using TensorFlow backend.


In [2]:
nb_classes = 1  # number of classes
img_width, img_height = 64, 64  # change based on the shape/structure of your images
batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 50  # number of iteration the algorithm gets trained.
learn_rate = 1e-5  # sgd learning rate

# Dir

In [3]:
train_dir = '/home/www/fake_detection/StyleGAN/train'
validation_dir = '/home/www/fake_detection/StyleGAN/validation'
test50_dir = '/home/www/fake_detection/StyleGAN/test'

# ShallowNet

In [4]:
# ShallowNet V3
img_input = Input(shape=(img_height, img_width, 3))

# block 1
x = Conv2D(32, (5, 5), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(img_input)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)
# x = Dropout(0.25)(x)

# block 2
x = Conv2D(64, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)
x = Conv2D(64, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)
x = Conv2D(64, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)
# x = Dropout(0.25)(x)

# block 3
x = Conv2D(128, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
# x = Dropout(0.25)(x)

# block 4
x_flatten = Flatten()(x)
x_de = Dense(1024, kernel_regularizer=regularizers.l2(0.0001))(x_flatten)
x_rl = Activation('relu')(x_de)
x_bn = BatchNormalization()(x_rl)
x_dp = Dropout(0.25)(x_bn)

x_sig = Dense(2, activation=None)(x_bn)
out = Activation('sigmoid')(x_sig)

model = Model(img_input, out)
print("Loaded Model from disk")
print(model.summary())

W1006 14:11:19.379235 10060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Loaded Model from disk
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        1056      
_________________________________________________________________
activation_2 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        1056      
_________________________________________________________________
activation_3 (Activation)    (None, 

In [5]:
# model.load_weights("/home/kim3/fake_detection/model/pretrained/shallownetv3.hd5f")
# print(model.get_weights()[0][0])

In [6]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

22


# Data generator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

Found 70640 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.
Found 60000 images belonging to 2 classes.


In [8]:
# callback_list = [EarlyStopping(monitor='val_accuracy', patience=10),
#                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]
# history = model.fit_generator(train_generator,
#                             steps_per_epoch=200,
#                             epochs=100,
#                             validation_data=validation_generator,
#                             validation_steps=len(validation_generator),
#                             callbacks=callback_list)

In [9]:
# model.save('/home/www/fake_detection/model/style_shallownetv3.h5')

In [10]:
# model = load_model('/home/www/fake_detection/model/style_shallownetv3.h5')

In [11]:
# output = model.predict_generator(test50_generator, steps=len(test50_generator), verbose=1)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# print(test50_generator.class_indices)
# print(output)

In [12]:
# output_score50 = []
# output_class50 = []
# answer_class50 = []
# answer_class50_1 =[]

# for i in trange(len(test50_generator)):
#     output50 = model.predict_on_batch(test50_generator[i][0])
#     output_score50.append(output50)
#     answer_class50.append(test50_generator[i][1])
    
# output_score50 = np.concatenate(output_score50)
# answer_class50 = np.concatenate(answer_class50)

# output_class50 = np.argmax(output_score50, axis=1)
# answer_class50_1 = np.argmax(answer_class50, axis=1)

# print(output_class50)
# print(answer_class50_1)

In [13]:
# cm50 = confusion_matrix(answer_class50_1, output_class50)
# report50 = classification_report(answer_class50_1, output_class50)

# recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
# fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

# fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
# eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
# thresh50 = interp1d(fpr50, thresholds50)(eer50)

# print(report50)
# print(cm50)
# print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
# print(thresh50)
# print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))

In [14]:
def cutout(img):
    """
    # Function: RandomCrop (ZeroPadded (4, 4)) + random occulusion image
    # Arguments:
        img: image
    # Returns:
        img
    """
    height = img.shape[0]
    width = img.shape[1]
    channels = img.shape[2]
    MAX_CUTS = 3 # chance to get more cuts
    MAX_LENGTH_MUTIPLIER = 10 # chance to get larger cuts
    # 16 for cifar10, 8 for cifar100
    
    # Zero-padded (4, 4)
#     img = np.pad(img, ((4,4),(4,4),(0,0)), mode='constant', constant_values=(0))
    
#     # random-crop 64x64
#     dy, dx = height, width
#     x = np.random.randint(0, width - dx + 1)
#     y = np.random.randint(0, height - dy + 1)
#     img = img[y:(y+dy), x:(x+dx)]
    
#     mean norm
#     mean = img.mean(keepdims=True)
#     img -= mean

    img *= 1./255
    
    mask = np.ones((height, width, channels), dtype=np.float32)
    nb_cuts = np.random.randint(0, MAX_CUTS + 1)
    
    # cutout
    for i in range(nb_cuts):
        y = np.random.randint(height)
        x = np.random.randint(width)
        length = 4 * np.random.randint(1, MAX_LENGTH_MUTIPLIER+1)
        
        y1 = np.clip(y-length//2, 0, height)
        y2 = np.clip(y+length//2, 0, height)
        x1 = np.clip(x-length//2, 0, width)
        x2 = np.clip(x+length//2, 0, width)
        
        mask[y1:y2, x1:x2, :] = 0.
    
    img = img * mask
    return img

In [15]:
class ReLU6(Layer):
    def __init__(self):
        super().__init__(name="ReLU6")
        self.relu6 = ReLU(max_value=6, name="ReLU6")

    def call(self, input):
        return self.relu6(input)


class HardSigmoid(Layer):
    def __init__(self):
        super().__init__()
        self.relu6 = ReLU6()

    def call(self, input):
        return self.relu6(input + 3.0) / 6.0


class HardSwish(Layer):
    def __init__(self):
        super().__init__()
        self.hard_sigmoid = HardSigmoid()

    def call(self, input):
        return input * self.hard_sigmoid(input)
    
class Attention(Layer):
    def __init__(self, ch, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.channels = ch
        self.filters_f_g = self.channels // 8
        self.filters_h = self.channels

    def build(self, input_shape):
        kernel_shape_f_g = (1, 1) + (self.channels, self.filters_f_g)
        print(kernel_shape_f_g)
        kernel_shape_h = (1, 1) + (self.channels, self.filters_h)

        # Create a trainable weight variable for this layer:
        self.gamma = self.add_weight(name='gamma', shape=[1], initializer='zeros', trainable=True)
        self.kernel_f = self.add_weight(shape=kernel_shape_f_g,
                                        initializer='glorot_uniform',
                                        name='kernel_f')
        self.kernel_g = self.add_weight(shape=kernel_shape_f_g,
                                        initializer='glorot_uniform',
                                        name='kernel_g')
        self.kernel_h = self.add_weight(shape=kernel_shape_h,
                                        initializer='glorot_uniform',
                                        name='kernel_h')
        self.bias_f = self.add_weight(shape=(self.filters_f_g,),
                                      initializer='zeros',
                                      name='bias_F')
        self.bias_g = self.add_weight(shape=(self.filters_f_g,),
                                      initializer='zeros',
                                      name='bias_g')
        self.bias_h = self.add_weight(shape=(self.filters_h,),
                                      initializer='zeros',
                                      name='bias_h')
        super(Attention, self).build(input_shape)
        # Set input spec.
        self.input_spec = InputSpec(ndim=4,
                                    axes={3: input_shape[-1]})
        self.built = True


    def call(self, x):
        def hw_flatten(x):
            return K.reshape(x, shape=[K.shape(x)[0], K.shape(x)[1]*K.shape(x)[2], K.shape(x)[-1]])

        f = K.conv2d(x,
                     kernel=self.kernel_f,
                     strides=(1, 1), padding='same')  # [bs, h, w, c']
        f = K.bias_add(f, self.bias_f)
        g = K.conv2d(x,
                     kernel=self.kernel_g,
                     strides=(1, 1), padding='same')  # [bs, h, w, c']
        g = K.bias_add(g, self.bias_g)
        h = K.conv2d(x,
                     kernel=self.kernel_h,
                     strides=(1, 1), padding='same')  # [bs, h, w, c]
        h = K.bias_add(h, self.bias_h)

        s = tf.matmul(hw_flatten(g), hw_flatten(f), transpose_b=True)  # # [bs, N, N]

        beta = K.softmax(s, axis=-1)  # attention map

        o = K.batch_dot(beta, hw_flatten(h))  # [bs, N, C]

        o = K.reshape(o, shape=K.shape(x))  # [bs, h, w, C]
        x = self.gamma * o + x

        return x

    def compute_output_shape(self, input_shape):
        return input_shape

In [16]:
ft_dir = '/home/www/fake_detection/StyleGAN/fine_tune'
train_gen_aug = ImageDataGenerator(shear_range=0, 
                               zoom_range=0.2,
                               rotation_range=0.2,
                               width_shift_range=2., 
                               height_shift_range=2.,
                               horizontal_flip=True,
                               zca_whitening=False,
                               fill_mode='nearest',
                               preprocessing_function=cutout)

test_datagen = ImageDataGenerator(rescale=1./255)

ft_gen = train_gen_aug.flow_from_directory(ft_dir,
                                            target_size=(img_height, img_width),
                                            batch_size=128,
                                            shuffle=True,
                                            class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    class_mode='categorical')

Found 1998 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.
Found 60000 images belonging to 2 classes.


In [17]:
model_ft = load_model('/home/www/fake_detection/model/style_shallownetv3.h5')
for i in range(7):
    model_ft.layers.pop()
im_in = Input(shape=(img_width, img_height, 3))

base_model = Model(img_input, x)
base_model.set_weights(model_ft.get_weights())
# for i in range(len(base_model.layers) - 0):
#     base_model.layers[i].trainable = False
    
x1 = base_model(im_in) # (12, 12, 32)
########### Mobilenet block bneck 3x3 (32 --> 128) #################
expand1 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(x1)
expand1 = BatchNormalization()(expand1)
expand1 = HardSwish()(expand1)
dw1 = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand1)
dw1 = BatchNormalization()(dw1)
se_gap1 = GlobalAveragePooling2D()(dw1)
se_gap1 = Reshape([1, 1, -1])(se_gap1)
se1 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap1)
se1 = Activation('relu')(se1)
se1 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se1)
se1 = HardSigmoid()(se1)
se1 = Multiply()([expand1, se1])
project1 = HardSwish()(se1)
project1 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project1)
project1 = BatchNormalization()(project1)

########### Mobilenet block bneck 5x5 (128 --> 128) #################
expand2 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(project1)
expand2 = BatchNormalization()(expand2)
expand2 = HardSwish()(expand2)
dw2 = DepthwiseConv2D(kernel_size=(5,5), strides=(1,1), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand2)
dw2 = BatchNormalization()(dw2)
se_gap2 = GlobalAveragePooling2D()(dw2)
se_gap2 = Reshape([1, 1, -1])(se_gap2)
se2 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap2)
se2 = Activation('relu')(se2)
se2 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se2)
se2 = HardSigmoid()(se2)
se2 = Multiply()([expand2, se2])
project2 = HardSwish()(se2)
project2 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project2)
project2 = BatchNormalization()(project2)
project2 = Add()([project1, project2])

########### Mobilenet block bneck 5x5 (128 --> 128) #################
expand3 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(project2)
expand3 = BatchNormalization()(expand3)
expand3 = HardSwish()(expand3)
dw3 = DepthwiseConv2D(kernel_size=(5,5), strides=(1,1), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand3)
dw3 = BatchNormalization()(dw3)
se_gap3 = GlobalAveragePooling2D()(dw3)
se_gap3 = Reshape([1, 1, -1])(se_gap3)
se3 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap3)
se3 = Activation('relu')(se3)
se3 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se3)
se3 = HardSigmoid()(se3)
se3 = Multiply()([expand3, se3])
project3 = HardSwish()(se3)
project3 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project3)
project3 = BatchNormalization()(project3)
project3 = Add()([project2, project3])


########## Classification ##########
x2 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project1)
x2 = BatchNormalization()(x2)
x2 = HardSwish()(x2)
x2 = GlobalAveragePooling2D()(x2)
x2 = Reshape([1, 1, -1])(x2)


# ######### Image Attention Model #########
### Block 1 ###
x3 = SeparableConv2D(32, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(im_in)
x3 = BatchNormalization()(x3)
x3 = HardSwish()(x3)
x3 = Attention(32)(x3)

### Block 2 ###
x4 = SeparableConv2D(64, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(x3)
x4 = BatchNormalization()(x4)
x4 = HardSwish()(x4)
x4 = Attention(64)(x4)

### Block 3 ###
x5 = SeparableConv2D(128, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(x4)
x5 = BatchNormalization()(x5)
x5 = HardSwish()(x5)
x5 = Attention(128)(x5)

### final stage ###
x6 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(x5)
x6 = BatchNormalization()(x6)
x6 = HardSwish()(x6)
x6 = GlobalAveragePooling2D()(x6)
x6 = Reshape([1, 1, -1])(x6)

# ######## final addition #########

x2 = Add()([x2, x6])
x2 = Conv2D(2, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5))(x2)
x2 = Activation('softmax')(x2)
x2 = Lambda(lambda x: K.squeeze(x, 1))(x2)
x2 = Lambda(lambda x: K.squeeze(x, 1))(x2)

model_top = Model(inputs=im_in, outputs=x2)
model_top.summary()

W1006 14:11:44.271486 10060 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1006 14:11:45.347609 10060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



(1, 1, 32, 4)
(1, 1, 64, 8)
(1, 1, 128, 16)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 12, 12, 32)   109344      input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 12, 12, 576)  18432       model_2[1][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 12, 12, 576)  2304        conv2d_9[0][0]                   
________________________________________________

In [ ]:
# optimizer = SGD(lr=1e-3, momentum=0.9, nesterov=True)
optimizer = Adam()
model_top.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
callback_list = [EarlyStopping(monitor='val_acc', patience=30), 
                 ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.5), cooldown=0, patience=5, min_lr=0.5e-5)]
output = model_top.fit_generator(ft_gen, steps_per_epoch=200, epochs=300,
                                  validation_data=validation_generator, validation_steps=validation_generator.samples//batch_size, callbacks=callback_list)

Epoch 1/300
200/200 [==============================] - 469s 2s/step - loss: 0.7192 - acc: 0.5194 - val_loss: 0.6590 - val_acc: 0.4987
Epoch 2/300
200/200 [==============================] - 416s 2s/step - loss: 0.7027 - acc: 0.5403 - val_loss: 0.5653 - val_acc: 0.4973
Epoch 3/300
200/200 [==============================] - 354s 2s/step - loss: 0.6937 - acc: 0.5601 - val_loss: 0.5709 - val_acc: 0.5171
Epoch 4/300
200/200 [==============================] - 301s 2s/step - loss: 0.6820 - acc: 0.5792 - val_loss: 1.9484 - val_acc: 0.5090
Epoch 5/300
200/200 [==============================] - 421s 2s/step - loss: 0.6706 - acc: 0.5980 - val_loss: 0.7074 - val_acc: 0.5260
Epoch 6/300
200/200 [==============================] - 256s 1s/step - loss: 0.6620 - acc: 0.6134 - val_loss: 1.8464 - val_acc: 0.5181
Epoch 7/300
200/200 [==============================] - 411s 2s/step - loss: 0.6503 - acc: 0.6255 - val_loss: 1.4501 - val_acc: 0.5298
Epoch 8/300
200/200 [==============================] - 304s 2s

In [ ]:
output_score50 = []
output_class50 = []
answer_class50 = []
answer_class50_1 =[]

for i in trange(len(test50_generator)):
    output50 = model_top.predict_on_batch(test50_generator[i][0])
    output_score50.append(output50)
    answer_class50.append(test50_generator[i][1])
    
output_score50 = np.concatenate(output_score50)
answer_class50 = np.concatenate(answer_class50)

output_class50 = np.argmax(output_score50, axis=1)
answer_class50_1 = np.argmax(answer_class50, axis=1)

print(output_class50)
print(answer_class50_1)

In [ ]:
cm50 = confusion_matrix(answer_class50_1, output_class50)
report50 = classification_report(answer_class50_1, output_class50)

recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
thresh50 = interp1d(fpr50, thresholds50)(eer50)

print(report50)
print(cm50)
print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
print(thresh50)
print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))